<a href="https://colab.research.google.com/github/DaffaPoseidon/PythonJobsheet7Semester4/blob/main/DecisionTree_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

In [4]:
waktu = 'PENDEK, PENDEK, PENDEK, PENDEK, PENDEK, PANJANG, PANJANG, PENDEK, PANJANG, PENDEK, PANJANG, PANJANG'.split(',')
paket = 'BESAR, KECIL, KECIL, KECIL, KECIL, BESAR, KECIL, BESAR, KECIL, KECIL, BESAR, KECIL'.split(',')
prioritas = 'RENDAH, RENDAH, TINGGI, RENDAH, TINGGI, RENDAH, TINGGI, RENDAH, TINGGI, TINGGI, TINGGI, TINGGI'.split(',')
frekuensi = 'SEDANG, TINGGI, SEDANG, TINGGI, SEDANG, SEDANG, TINGGI, SEDANG, RENDAH, TINGGI, TINGGI, RENDAH'.split(',')
gangguan = 'GANGGUAN, NORMAL, GANGGUAN, NORMAL, NORMAL, NORMAL, GANGGUAN, NORMAL, NORMAL, NORMAL, NORMAL, NORMAL'.split(',')

dataset ={'WAKTU':waktu, 'PAKET':paket, 'FREKUENSI':frekuensi, 'PRIORITAS':prioritas, 'GANGGUAN':gangguan }
df = pd.DataFrame(dataset,columns=[ 'WAKTU', 'PAKET', 'FREKUENSI', 'GANGGUAN', 'PREDIKSI'])

In [5]:
print("Hasil Prediksi Data Training : ")
df.head(12)

Hasil Prediksi Data Training : 


,WAKTU,PAKET,FREKUENSI,GANGGUAN,PREDIKSI
0,PENDEK,BESAR,SEDANG,GANGGUAN,NaN
1,PENDEK,KECIL,TINGGI,NORMAL,NaN
2,PENDEK,KECIL,SEDANG,GANGGUAN,NaN
3,PENDEK,KECIL,TINGGI,NORMAL,NaN
4,PENDEK,KECIL,SEDANG,NORMAL,NaN
5,PANJANG,BESAR,SEDANG,NORMAL,NaN
6,PANJANG,KECIL,TINGGI,GANGGUAN,NaN
7,PENDEK,BESAR,SEDANG,NORMAL,NaN
8,PANJANG,KECIL,RENDAH,NORMAL,NaN
9,PENDEK,KECIL,TINGGI,NORMAL,NaN


In [6]:
waktu = 'PENDEK, PENDEK, PENDEK, PENDEK, PENDEK, PANJANG, PANJANG, PENDEK, PANJANG, PENDEK, PANJANG, PANJANG'.split(',')
paket = 'BESAR, KECIL, KECIL, KECIL, KECIL, BESAR, KECIL, BESAR, KECIL, KECIL, BESAR, KECIL'.split(',')
prioritas = 'RENDAH, RENDAH, TINGGI, RENDAH, TINGGI, RENDAH, TINGGI, RENDAH, TINGGI, TINGGI, TINGGI, TINGGI'.split(',')
frekuensi = 'SEDANG, TINGGI, SEDANG, TINGGI, SEDANG, SEDANG, TINGGI, SEDANG, RENDAH, TINGGI, TINGGI, RENDAH'.split(',')
gangguan = 'GANGGUAN, NORMAL, GANGGUAN, NORMAL, NORMAL, NORMAL, GANGGUAN, NORMAL, NORMAL, NORMAL, NORMAL, NORMAL'.split(',')
prediksi = 'YA, TIDAK, YA, TIDAK, TIDAK, TIDAK, TIDAK, YA, TIDAK, TIDAK, TIDAK, TIDAK'.split(',')

dataset ={'WAKTU':waktu, 'PAKET':paket, 'FREKUENSI':frekuensi, 'PRIORITAS':prioritas, 'GANGGUAN':gangguan, 'PREDIKSI':prediksi }
df = pd.DataFrame(dataset,columns=[ 'WAKTU', 'PAKET', 'PRIORITAS', 'FREKUENSI', 'GANGGUAN', 'PREDIKSI'])

In [7]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

def ig(e_dataset,e_attr):
    return(e_dataset-e_attr)

def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]
  
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['GANGGUAN'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

In [8]:
print("Hasil Predikdi Data Prediksi: ")
df.head(12)

Hasil Predikdi Data Prediksi: 


,WAKTU,PAKET,PRIORITAS,FREKUENSI,GANGGUAN,PREDIKSI
0,PENDEK,BESAR,RENDAH,SEDANG,GANGGUAN,YA
1,PENDEK,KECIL,RENDAH,TINGGI,NORMAL,TIDAK
2,PENDEK,KECIL,TINGGI,SEDANG,GANGGUAN,YA
3,PENDEK,KECIL,RENDAH,TINGGI,NORMAL,TIDAK
4,PENDEK,KECIL,TINGGI,SEDANG,NORMAL,TIDAK
5,PANJANG,BESAR,RENDAH,SEDANG,NORMAL,TIDAK
6,PANJANG,KECIL,TINGGI,TINGGI,GANGGUAN,TIDAK
7,PENDEK,BESAR,RENDAH,SEDANG,NORMAL,YA
8,PANJANG,KECIL,TINGGI,RENDAH,NORMAL,TIDAK
9,PENDEK,KECIL,TINGGI,TINGGI,NORMAL,TIDAK


In [9]:
import pprint
T = buildTree(df)

entropy_node = find_entropy(df)
print("Entropy seluruh dataset: " ,entropy_node)

a_entropy = {k:find_entropy_attribute(df,k) for k in df.keys()[:-1]}
print("Entropy  setiap atribut: ")
print(a_entropy)

k = 0
a_entropy[k] = {k:ig(entropy_node,a_entropy[k]) for k in a_entropy}
print("\nInformasi Persentase gangguan: ")
print(a_entropy[0])

print("\nHasil Prediksi Decision Tree berdasarkan Waktu: ")
pprint.pprint(T)

Entropy seluruh dataset:  1.0408520829727552
Entropy  setiap atribut: 
{'WAKTU': 0.45914791702724433, 'PAKET': 0.5919502539800194, 'PRIORITAS': 0.6155684956594014, 'FREKUENSI': 0.3333333333333329, 'GANGGUAN': 0.5441104177484003}

Informasi Persentase gangguan: 
{'WAKTU': 0.5817041659455109, 'PAKET': 0.4489018289927358, 'PRIORITAS': 0.4252835873133538, 'FREKUENSI': 0.7075187496394223, 'GANGGUAN': 0.4967416652243549}

Hasil Prediksi Decision Tree berdasarkan Waktu: 
{'FREKUENSI': {' RENDAH': ' NORMAL',
               ' SEDANG': {'WAKTU': {' PANJANG': ' NORMAL',
                                     ' PENDEK': {'PAKET': {' BESAR': ' NORMAL',
                                                           ' KECIL': {'GANGGUAN': {' GANGGUAN': ' '
                                                                                                'GANGGUAN',
                                                                                   ' NORMAL': ' '
                                                